In [ ]:
# 제목, 본문, 댓글, 시간 빈 list 생성

titles = [] #제목
reviews = [] #리뷰
comments = [] #댓글
date = [] # 시간

count_view=0


# 1. 첫번째 게시물 클릭 (선택자 입력_여기서는 xpath copy 해줌)
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]').click()
#driver.find_element('xpath','//*[@id="main-area"]/div[5]/table/tbody/tr[5]/td[1]/div[2]/div/a[1]').click()


# 2. for 문으로 지정한 게시물 개수까지 돌림

for i in range(1): #새로 창열기
    for i in range(30): # 한 번에 몇 개의 개시물을 크롤링 할 것인지 지정        
        driver.implicitly_wait(30) 
        
        # 제목 수집
        time.sleep(random.uniform(0.5,1)) # 0.5~1초 랜덤한 시간으로 쉬어줘
        title = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)    

       
        # 글 본문 수집 (3가지 경우 존재)
        # 본문의 경우 선택자 1을 확인하고 코드를 작성했으나, 또 다른 선택자가 존재하여 선택자 2를 다시 try해주도록 만들었습니다.
        try: 
            content = driver.find_element(By.CSS_SELECTOR,'div.se-module.se-module-text').text # 선택자1 ('div.se-module.se-module-text')
            reviews.append(content)       
            
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR,'div.ContentRenderer').text # 선택자2 ('div.ContentRenderer')
                reviews.append(content)
            except:
                content= [] # 본문이 사진으로만 구성 (담을게 없어서 빈 리스트 생성해서 append 시켜줌)
                reviews.append(content)


         # 댓글 수집 (2가지 경우 존재)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span',class_='text_comment') # find_all = text_comment 전부 다 찾아줘
        if len(iscomment) == 0: # 댓글 없는 경우
            box = []

        else: # 댓글 있는 경우
                
            WebDriverWait(driver, 15).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "text_comment")) )
                # 웹드라이버를 통해 브라우져에서 최대 15초 기다려주고, text_comment(댓글 선택자) 기준으로 나올때 까지 기다려줘
            
            soup = bs(driver.page_source, 'lxml')  
            iscomment = soup.find_all('span',class_='text_comment') 
            box = []
            for i in iscomment:
                box.append([i.get_text()]) # iscome에 할당한 text_comment에서 text만 box에 append

        comments.append(box)

        # 시간 수집
        times = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date.append(times) 
        
        
        # count view 크롤링 진행상황 보기 (50개 마다 출력)
        count_view = count_view + 1
        if count_view % 50 ==0: #50으로 나누었을 때 0이 되면 게시물 크롤링 완료라고 보여줘
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass
        

        
        
        # 다음글 클릭 (다음글 버튼 클릭해서 연속적으로 크롤링, 첫번째글->두번째글->세번째글...)
        try:
            driver.find_element(By.CSS_SELECTOR,"#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span").click()
            driver.implicitly_wait(28)
            
        except: # 다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            
            current_url = driver.current_url # 다음글 버튼이 사라져서 크롤링이 멈춘 url 가지고 오기
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  


            ##재로그인
            # 네이버 로그인 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(9)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)


            # '로그인' 버튼 클릭
            driver.find_element('id','log.login').click()
            time.sleep(1)

            # 멈친 부분의 링크 가지고오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)    

                  
    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
    link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")
    
    #현재 크롬창 닫기
    driver.close()
    
    
    # 크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  
    
       
    ##재로그인
    # 네이버 로그인 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(9)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    # '로그인' 버튼 클릭
    driver.find_element('id','log.login').click()
    time.sleep(1)
    
    #다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')


# 크롬 창 닫기
driver.quit()